# Logistic regression

The method of logistic regression is a classical machine learning classifier and meant to serve as a baseline for later comparison with the three GNNs. This classifier does not take the graph structure into account.

In [1]:
import numpy as np
import pandas as pd

import ipynb
from ipynb.fs.defs.dataset_creation import NodeClassificationDataset

import json
import os
import os.path as osp
import time

from sklearn.linear_model import LogisticRegression

import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.data import Dataset, download_url
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

## Define helper functions

In [2]:
def reverse(tuples):
    """
    Reverse a 2-tuple.
    """
    new_tup = tuples[::-1]
    
    return new_tup

In [3]:
def mean(dict_values):
    """
    Get mean of dictionary values.
    """
    return np.mean(np.array(list(dict_values)))

In [4]:
def std(dict_values):
    """
    Get standard deviation of dictionary values.
    """
    return np.std(np.array(list(dict_values)))

In [5]:
def filter_best_hyperparameters(df, variable):
    """
    Filter the results data frame for the best hyperparameter combination(s).
    """
    return df[df[variable] == df[variable].max()]

In [6]:
def print_best_hyperparameters(df, variable):
    """
    Print best hyperparameter combination(s).
    """
    df_best = df[df[variable] == df[variable].max()]
    print('The best hyperparameter combination/s is/are:\n\t', 
          '\n\t '.join([str(df_best['parameters'].iloc[i]) for i in range(len(df_best))]),
          '\nThe resulting mean validation accuracy is:', df_best[variable].unique()[0])

## Hyperparameter tuning

We use the [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) of logistic regression `LogisticRegression`. The article [Tune Hyperparameters for Classification Machine Learning Algorithms](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) gives a good overview of hyperparameter tuning for logistic regression.

With logistic regression, the main parameters to tune are:
* `penalty`: specifies the norm of the penalty. `'none'`: no penalty is added. `'l2'`: add an L2 penalty term and it is the default choice. `'l1'`: add an L1 penalty term. `'elasticnet'`: both L1 and L2 penalty terms are added.
* `C`: inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
* `solver`: algorithm to use in the optimization problem. Possible solvers: 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'. Default is 'lbfgs'. To choose a solver, you might want to consider the following aspects. For small datasets, 'liblinear' is a good choice, whereas 'sag' and 'saga' are faster for large ones. For multiclass problems, only 'newton-cg', 'sag', 'saga' and 'lbfgs' handle multinomial loss. 'liblinear' is limited to one-versus-rest schemes.

There are still other hyperparameters that can be tuned. We do not tune them and instead rely on their default values provided in scikit-learn. We now perform the hyperparameter tuning using loops.

In [7]:
# Create results folder
route0 = "./results"

if not os.path.exists(route0):
    os.mkdir(route0)

In [16]:
start_time = time.time()

lrc_results = pd.DataFrame(columns = ['parameters', 'train_accs', 'val_accs', 'test_accs', 'mean_train_acc', 'mean_val_acc', 
                                      'mean_test_acc', 'std_train_acc', 'std_val_acc', 'std_test_acc'])

seeds = [12345, 23456, 34567, 45678, 56789, 54321, 65432, 76543, 87654, 98765]

for penalty in ['l1', 'l2', 'elasticnet', 'none']:
    for C in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
        for solver in ['newton-cg', 'lbfgs', 'sag', 'saga']: 
            if (solver == 'newton-cg' or solver == 'lbfgs' or solver == 'sag') and (penalty == 'l1' or penalty == 'elasticnet'): # newton-cg only works with l2 and none
                continue 

            print(f'====================================================================================================\nParameter combination: {{penalty: {penalty}, C: {C}, solver: {solver}}}\n====================================================================================================')

            parameter_dict = dict({'penalty': penalty, 'C': C, 'solver': solver})

            train_acc_dict = dict() 
            val_acc_dict = dict()
            test_acc_dict = dict()

            for i, seed in enumerate(seeds):

                print(f'==================================================\nRun {i+1} with seed {seed}:\n==================================================')

                torch_geometric.seed_everything(seed)

                dataset = NodeClassificationDataset(root='data/', transform=RandomNodeSplit(split="random", num_train_per_class = 10, num_val = 60, num_test = 129))
                data = dataset[0]

                # Create train, validation and test data
                x_train = data.x[data.train_mask].numpy()
                x_val = data.x[data.val_mask].numpy()
                x_test = data.x[data.test_mask].numpy()

                y_train = data.y[data.train_mask].numpy()
                y_val = data.y[data.val_mask].numpy()
                y_test = data.y[data.test_mask].numpy()

                lrc = LogisticRegression(penalty=penalty, C=C, solver=solver, l1_ratio=0.5) 

                lrc.fit(x_train, y_train)

                # Get predictions on validation and test set
                train_pred = lrc.predict(x_train)
                val_pred = lrc.predict(x_val)
                test_pred = lrc.predict(x_test)

                # Get mean accuracies
                train_acc = lrc.score(x_train, y_train)
                val_acc = lrc.score(x_val, y_val)
                test_acc = lrc.score(x_test, y_test)

                print(f"Train mean accuracy: {train_acc} \nValidation mean accuracy: {val_acc} \nTest mean accuracy: {test_acc}")

                train_acc_dict.update({f"{i+1}": train_acc})
                val_acc_dict.update({f"{i+1}": val_acc})
                test_acc_dict.update({f"{i+1}": test_acc})

            data_concat = [{'parameters': parameter_dict, 'train_accs': train_acc_dict, 'val_accs': val_acc_dict, 
                            'test_accs': test_acc_dict, 'mean_train_acc': mean(train_acc_dict.values()), 
                            'mean_val_acc': mean(val_acc_dict.values()), 'mean_test_acc': mean(test_acc_dict.values()), 
                            'std_train_acc': std(train_acc_dict.values()),'std_val_acc': std(val_acc_dict.values()), 
                            'std_test_acc': std(test_acc_dict.values())}]
            df_concat = pd.DataFrame(data_concat)
            lrc_results = pd.concat([lrc_results, df_concat], ignore_index=True)

print('====================================================================================================',
      '\n====================================================================================================', 
      '\n====================================================================================================')

print("saving file corresponding to lrc_results.pkl")
lrc_results.to_pickle(f"{route0}/lrc_results.pkl")

end_time = time.time()
duration = end_time - start_time
print(f"{int(np.floor(duration/3600))} hours {(duration-3600*np.floor(duration/3600))/60:.4} minutes")

Parameter combination: {penalty: l1, C: 0.001, solver: saga}
Run 1 with seed 12345:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.16666666666666666 
Test mean accuracy: 0.26356589147286824
Run 2 with seed 23456:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.23333333333333334 
Test mean accuracy: 0.32558139534883723
Run 3 with seed 34567:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.05 
Test mean accuracy: 0.023255813953488372
Run 4 with seed 45678:


Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1

Train mean accuracy: 0.25 
Validation mean accuracy: 0.5 
Test mean accuracy: 0.4108527131782946
Run 5 with seed 56789:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.35 
Test mean accuracy: 0.2713178294573643
Run 6 with seed 54321:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.3 
Test mean accuracy: 0.29457364341085274
Run 7 with seed 65432:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.2 
Test mean accuracy: 0.24806201550387597
Run 8 with seed 76543:


Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1

Train mean accuracy: 0.25 
Validation mean accuracy: 0.3 
Test mean accuracy: 0.29457364341085274
Run 9 with seed 87654:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.016666666666666666 
Test mean accuracy: 0.03875968992248062
Run 10 with seed 98765:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.03333333333333333 
Test mean accuracy: 0.031007751937984496
Parameter combination: {penalty: l1, C: 0.01, solver: saga}
Run 1 with seed 12345:
Train mean accuracy: 0.25 
Validation mean accuracy: 0.16666666666666666 
Test mean accuracy: 0.26356589147286824
Run 2 with seed 23456:


...

Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn("l1_ratio parameter is only used when penalty is "
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn("l1_ratio parameter is only used when penalty is "
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarn

Train mean accuracy: 1.0 
Validation mean accuracy: 0.8166666666666667 
Test mean accuracy: 0.9224806201550387
Run 7 with seed 65432:
Train mean accuracy: 1.0 
Validation mean accuracy: 0.8333333333333334 
Test mean accuracy: 0.8837209302325582
Run 8 with seed 76543:


Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn("l1_ratio parameter is only used when penalty is "
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn("l1_ratio parameter is only used when penalty is "


Train mean accuracy: 1.0 
Validation mean accuracy: 0.8333333333333334 
Test mean accuracy: 0.8914728682170543
Run 9 with seed 87654:
Train mean accuracy: 1.0 
Validation mean accuracy: 0.9166666666666666 
Test mean accuracy: 0.7906976744186046
Run 10 with seed 98765:
Train mean accuracy: 1.0 
Validation mean accuracy: 0.85 
Test mean accuracy: 0.8217054263565892
saving file corresponding to lrc_results.pkl
0 hours 1.025 minutes


C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
Processing...
Done!
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn("l1_ratio parameter is only used when penalty is "
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\bwank\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  

In [8]:
# Look at results
lrc_results = pd.read_pickle(f"{route0}/lrc_results.pkl") 
lrc_results

,parameters,train_accs,val_accs,test_accs,mean_train_acc,mean_val_acc,mean_test_acc,std_train_acc,std_val_acc,std_test_acc
0,"{'penalty': 'l1', 'C': 0.001, 'solver': 'saga'}","{'1': 0.25, '2': 0.25, '3': 0.25, '4': 0.25, '...","{'1': 0.16666666666666666, '2': 0.233333333333...","{'1': 0.26356589147286824, '2': 0.325581395348...",0.2500,0.215000,0.220155,0.0000,0.147092,0.130877
1,"{'penalty': 'l1', 'C': 0.01, 'solver': 'saga'}","{'1': 0.25, '2': 0.25, '3': 0.25, '4': 0.25, '...","{'1': 0.16666666666666666, '2': 0.233333333333...","{'1': 0.26356589147286824, '2': 0.325581395348...",0.2500,0.215000,0.220155,0.0000,0.147092,0.130877
2,"{'penalty': 'l1', 'C': 0.1, 'solver': 'saga'}","{'1': 0.25, '2': 0.25, '3': 0.25, '4': 0.25, '...","{'1': 0.36666666666666664, '2': 0.066666666666...","{'1': 0.26356589147286824, '2': 0.015503875968...",0.2500,0.235000,0.241085,0.0000,0.149898,0.153850
3,"{'penalty': 'l1', 'C': 1, 'solver': 'saga'}","{'1': 0.95, '2': 0.925, '3': 0.975, '4': 0.925...","{'1': 0.8166666666666667, '2': 0.8166666666666...","{'1': 0.8837209302325582, '2': 0.8759689922480...",0.9325,0.840000,0.879070,0.0225,0.029059,0.019051
4,"{'penalty': 'l1', 'C': 10, 'solver': 'saga'}","{'1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': ...","{'1': 0.8333333333333334, '2': 0.95, '3': 0.86...","{'1': 0.8682170542635659, '2': 0.8914728682170...",0.9975,0.851667,0.877519,0.0075,0.048563,0.031162
...,...,...,...,...,...,...,...,...,...,...
65,"{'penalty': 'none', 'C': 100, 'solver': 'saga'}","{'1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': ...","{'1': 0.8333333333333334, '2': 0.9166666666666...","{'1': 0.875968992248062, '2': 0.88372093023255...",1.0000,0.835000,0.862791,0.0000,0.062561,0.042182
66,"{'penalty': 'none', 'C': 1000, 'solver': 'newt...","{'1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': ...","{'1': 0.8333333333333334, '2': 0.9333333333333...","{'1': 0.8604651162790697, '2': 0.8604651162790...",1.0000,0.840000,0.848837,0.0000,0.047842,0.028640
67,"{'penalty': 'none', 'C': 1000, 'solver': 'lbfgs'}","{'1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': ...","{'1': 0.8166666666666667, '2': 0.9333333333333...","{'1': 0.875968992248062, '2': 0.86046511627906...",1.0000,0.836667,0.860465,0.0000,0.060000,0.037015
68,"{'penalty': 'none', 'C': 1000, 'solver': 'sag'}","{'1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': ...","{'1': 0.8333333333333334, '2': 0.9333333333333...","{'1': 0.8837209302325582, '2': 0.8682170542635...",1.0000,0.831667,0.858140,0.0000,0.063004,0.044132


## Best hyperparameter combination

In [9]:
filter_best_hyperparameters(lrc_results, 'mean_val_acc')

,parameters,train_accs,val_accs,test_accs,mean_train_acc,mean_val_acc,mean_test_acc,std_train_acc,std_val_acc,std_test_acc
4,"{'penalty': 'l1', 'C': 10, 'solver': 'saga'}","{'1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': ...","{'1': 0.8333333333333334, '2': 0.95, '3': 0.86...","{'1': 0.8682170542635659, '2': 0.8914728682170...",0.9975,0.851667,0.877519,0.0075,0.048563,0.031162


In [10]:
print_best_hyperparameters(lrc_results, 'mean_val_acc')

The best hyperparameter combination/s is/are:
	 {'penalty': 'l1', 'C': 10, 'solver': 'saga'} 
The resulting validation accuracy is: 0.8516666666666668


We see that the best hyperparameter combination is given by:

* penalty = 'l1'
* C = 10
* solver = 'saga'

It gives a mean training accuracy of 0.9975, where the mean validation accuracy of 0.8517 and the mean test accuracy of 0.8775 are a little lower. The standard deviation of the training accuracies of 0.0075 is very low, whereas the standard deviations of the validation accuracies and the test accuracies are much higher, with values of 0.0486 and 0.0311, respectively.